In [1]:
import os
import sys
this = os.getcwd()
path = this[:this.rfind("/")]
if not path in sys.path: sys.path.append(path)
xgboost_path = path + "/xgboost_regressor"
promedios_path = path + "/promedio_zona"

import pandas as pd
import matplotlib
from datos import FEATURES_DISPONIBLES
from modelo import Modelo

pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
from xgboost_regressor.xgboost_predictor import XGBoostRegressor
from promedio_zona.promedio_zona import PromedioZona
from regresion_lineal.regresion_lineal import RegresionLineal
from mlp_regressor.mlp_regressor import MLP_Regressor

In [3]:
class EnsambleConcatenados(XGBoostRegressor):
    """
        Usa el resultado de las predicciones del modelo PromedioZona
        para entrenar y predecir con un XGBoostRegressor.
    """
    
    
    @Modelo.cronometrar()
    def __init__(self):
        self.modelo_promedios = PromedioZona()
        self.modelo_lineal = RegresionLineal()
        self.modelo_mlp = MLP_Regressor()
        super().__init__()        
        
    @Modelo.cronometrar()
    def cargar_datos(self):
        self.modelo_promedios.cargar_datos()
        self.modelo_lineal.cargar_datos()
        self.modelo_mlp.cargar_datos()
        super().cargar_datos()
    
    @Modelo.cronometrar()
    def entrenar(self):
        self.agregar_predicciones_modelo(self.modelo_lineal)
        self.agregar_predicciones_modelo(self.modelo_promedios)
        self.agregar_predicciones_modelo(self.modelo_mlp)
        super().entrenar()
    
    def agregar_predicciones_modelo(self, modelo):
        columna = 'prediccion_' + modelo.modelo
        modelo.entrenar()
        score = modelo.validar()
        print("Score individual {}: {}".format(modelo.modelo, score))
        predicciones_train =  modelo.predecir(modelo.train_data)
        predicciones_test = modelo.predecir(modelo.test_data)
        predicciones_submit = modelo.predecir(modelo.submit_data)
        self.train_data[columna] = predicciones_train['target']
        self.test_data[columna] = predicciones_test['target']
        self.submit_data[columna] = predicciones_submit['target']


In [4]:
ensamble = EnsambleConcatenados()

instanciar demoro 0.0 segundos
instanciar demoro 0.0 segundos
instanciar demoro 0.0 segundos
instanciar demoro 0.0 segundos
__init__ demoro 0.0 segundos


In [5]:
ensamble.cargar_datos()

cargar demoro 1.35 segundos
cargar demoro 64.67 segundos
cargar demoro 63.45 segundos
cargar demoro 64.63 segundos
cargar_datos demoro 228.83 segundos


In [6]:
ensamble.entrenar()

entrenar demoro 0.0 segundos
entrenar demoro 4.18 segundos
predecir demoro 0.26 segundos
puntuar demoro 0.0 segundos
validar demoro 0.26 segundos
Score individual RegresionLineal: 869898.1016045914
predecir demoro 0.62 segundos
predecir demoro 0.25 segundos
predecir demoro 0.24 segundos
entrenar demoro 0.0 segundos
entrenar demoro 0.22 segundos
predecir demoro 6.91 segundos
puntuar demoro 0.0 segundos
validar demoro 6.91 segundos
Score individual PromedioZona: 848990.0039788197
predecir demoro 21.41 segundos
predecir demoro 6.74 segundos
predecir demoro 6.95 segundos


/home/mharfuch/.local/share/virtualenvs/7506-TP-cfKyodU4/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


entrenar demoro 0.0 segundos
entrenar demoro 4903.51 segundos
predecir demoro 4.24 segundos
puntuar demoro 0.0 segundos
validar demoro 4.24 segundos
Score individual MLP_Regressor: 645079.0992153499
predecir demoro 10.87 segundos
predecir demoro 3.9 segundos
predecir demoro 3.83 segundos
entrenar demoro 0.0 segundos
entrenar demoro 947.95 segundos
entrenar demoro 5922.14 segundos


In [7]:
ensamble.validar()

predecir demoro 3.77 segundos
puntuar demoro 0.0 segundos
validar demoro 3.77 segundos


533776.5

In [8]:
predicciones = ensamble.predecir(ensamble.submit_data)

predecir demoro 3.68 segundos


In [10]:
comentario = "con regresion lineal mejorada- local 533776 - entrena en 6k segundos"
ensamble.presentar(predicciones, comentario)

guardar demoro 0.14 segundos


100%|██████████| 967k/967k [00:08<00:00, 122kB/s]  


submit demoro 11.06 segundos
presentar demoro 11.2 segundos


In [ ]:
predicciones = ensamble.predecir(ensamble.test_data)

In [ ]:
columnas_predictoras = ['target', 'prediccion_PromedioZona', 'prediccion_RegresionLineal']
for columna in columnas_predictoras:
    predicciones['diferencia_'+columna] = predicciones['precio'] - predicciones[columna]

In [ ]:
mejores_100 = predicciones.sort_values(by='diferencia_target').head(200)

In [ ]:
peores_100 = predicciones.sort_values(by='diferencia_target').tail(200)

In [ ]:
peores_100.describe()

In [ ]:
mejores_100.describe()